In [271]:
import pandas
import re

In [272]:
ground = pandas.read_json('data/input/draw.json')
ground = ground[['lSolutions', 'iIndex']]
ground.head(1)

,lSolutions,iIndex
0,[2.14285714286],397760


In [273]:
solved = pandas.read_json('data/output/solved/sample_0.jsonl', lines=True)
solved.head(1)

,question_id,id,object,created,model,usage,choices,question,n,temperature,solved,flag
0,113024,chatcmpl-7NHX754J9stbIGK5w2pqoUuT8P0nM,chat.completion,1685783297,gpt-3.5-turbo,"{'prompt_tokens': 407, 'completion_tokens': 11...","[{'message': {'role': 'assistant', 'content': ...",Extract the equations from this response.\nYou...,1,0.7,"{a: 20, b: 4}",OK


In [274]:
system = pandas.read_json('data/output/system_equations/sample_0.jsonl', lines=True)
system = system[['question_id', 'choices']]
system = system.rename(columns={'choices': 'system'})
system = system.set_index('question_id')
system.head(1)

,system
question_id,
74243,"[{'message': {'role': 'assistant', 'content': ..."


In [275]:
joined = pandas.merge(left=solved, right=ground, left_on='question_id', right_on='iIndex')
joined = joined.set_index('question_id').join(system)
joined.head(1)

,id,object,created,model,usage,choices,question,n,temperature,solved,flag,lSolutions,iIndex,system
question_id,,,,,,,,,,,,,,
337,chatcmpl-7NHX8nPNhsylPgMChy8Nn4EqdWrEl,chat.completion,1685783298,gpt-3.5-turbo,"{'prompt_tokens': 262, 'completion_tokens': 36...","[{'message': {'role': 'assistant', 'content': ...",Extract the equations from this response.\nYou...,1,0.7,{a: 2.40000000000000},OK,[2.4],337,"[{'message': {'role': 'assistant', 'content': ..."


In [276]:
def extract_actual_answer(answer):
    answer = re.sub(r'\s', "", answer)
    answer = re.sub(r'\s|.*####', "", answer)
    return eval(answer)

def remove_leading_zero(number : str):
    return re.sub(r'^0+', '', number)

def extract_chatgpt_answer(answer_text):
    answer_text = answer_text.lower()
    try:
        answers = re.findall(r'the\s*answer\s*is\s*.*', answer_text)[0]
        answers = re.findall(r'\d+\.?\d*', answers)

        to_return = []
        for answer in answers:
            try: to_return.append(round(eval(remove_leading_zero(answer))))
            except: pass
        return set(to_return)
    except:
        answers = re.findall(r'\d+\.?\d*', answer_text)
        if len(answers) == 0: return set()
        return set([answers[0]])

def extract_solved(solved):
    answers = re.findall(r'\d+\.?\d*', solved)
    return set([round(eval(answer), 2) for answer in answers])

def grade_response(row):
    solved = row['solved']
    solved = extract_solved(solved)
    system = row['system'][0]['message']['content']
    system = extract_chatgpt_answer(system)
    chatgpt_answer = extract_chatgpt_answer(row['question'])
    actual_answer = set(row['lSolutions'])

    A = 1 if len(chatgpt_answer.intersection(actual_answer)) > 0 else 0
    C = 1 if len(system.intersection(actual_answer)) > 0 else 0
    D = 1 if len(solved.intersection(actual_answer)) > 0 else 0

    row['A'] = A
    row['B'] = 1
    row['C'] = C
    row['D'] = D

    category_array = [A, C, D]

    offset = 8 if len(chatgpt_answer.intersection(solved)) > 0 else 0
    if category_array == [0, 0 , 0]: row['category'] = 1 + offset
    if category_array == [0, 0 , 1]: row['category'] = 2 + offset
    if category_array == [0, 1 , 0]: row['category'] = 3 + offset
    if category_array == [0, 1 , 1]: row['category'] = 4 + offset
    if category_array == [1, 0 , 0]: row['category'] = 5 + offset
    if category_array == [1, 0 , 1]: row['category'] = 6 + offset
    if category_array == [1, 1 , 0]: row['category'] = 7 + offset
    if category_array == [1, 1 , 1]: row['category'] = 8 + offset
    
    return row

In [277]:
joined = joined.apply(lambda row : grade_response(row), axis=1)

In [278]:
joined.head(1)

,id,object,created,model,usage,choices,question,n,temperature,solved,flag,lSolutions,iIndex,system,A,B,C,D,category
question_id,,,,,,,,,,,,,,,,,,,
337,chatcmpl-7NHX8nPNhsylPgMChy8Nn4EqdWrEl,chat.completion,1685783298,gpt-3.5-turbo,"{'prompt_tokens': 262, 'completion_tokens': 36...","[{'message': {'role': 'assistant', 'content': ...",Extract the equations from this response.\nYou...,1,0.7,{a: 2.40000000000000},OK,[2.4],337,"[{'message': {'role': 'assistant', 'content': ...",0,1,0,1,2


In [281]:
def label_category(row):

    if row['category'] == 1: 
        row['Equations'] = '''UNKNOWN'''
        row['Solving'] = '''FAIL'''
        row['Theory'] = '''Definitely cannot solve, may have got wrong rep.'''
        row['Understandable'] = '''YES'''
    if row['category'] == 2: 
        row['Equations'] = '''SUCCEED'''
        row['Solving'] = '''FAIL'''
        row['Theory'] = '''Gets equations right, but solves wrong'''

    return row

values = pandas.DataFrame(joined['category'].value_counts().sort_index())
values['category'] = values.index
values = values.apply(lambda row : label_category(row), axis=1)
values

,Equations,Solving,Understanable,category,count
category,,,,,
1,UNKNOWN,FAIL,"Definitely cannot solve, may have got wrong rep.",1.0,124.0
2,SUCCEED,FAIL,"Gets equations right, but solves wrong",2.0,56.0
3,NaN,NaN,NaN,3.0,51.0
4,NaN,NaN,NaN,4.0,71.0
5,NaN,NaN,NaN,5.0,7.0
6,NaN,NaN,NaN,6.0,2.0
7,NaN,NaN,NaN,7.0,21.0
8,NaN,NaN,NaN,8.0,9.0
9,NaN,NaN,NaN,9.0,52.0


,id,object,created,model,usage,choices,question,n,temperature,solved,flag,lSolutions,iIndex,system,A,B,C,D,category
question_id,,,,,,,,,,,,,,,,,,,
80313,chatcmpl-7NHZ38uOK9qNAXyI0QiRcF7k0yCxh,chat.completion,1685783417,gpt-3.5-turbo,"{'prompt_tokens': 287, 'completion_tokens': 44...","[{'message': {'role': 'assistant', 'content': ...",Extract the equations from this response.\nYou...,1,0.7,"{a: 6.00000000000000, b: 6.00000000000000, c: ...",OK,"[16.0, 8.0]",80313,"[{'message': {'role': 'assistant', 'content': ...",1,1,1,0,15
